# API Search - Make our bot to talk to any API

We have observed the remarkable synergy created by combining **GPT-4 with intelligent agents and detailed prompts**. This powerful combination has consistently delivered impressive results. To further capitalize on this capability, we should aim to integrate it with various systems through API communication. Essentially, we can develop within this notebook what is referred to in OpenAI's ChatGPT as 'GPTs.'

Envision a bot that seamlessly integrates with:

- **CRM Systems:** Including Dynamics, Salesforce, and HubSpot.
- **ERP Systems:** Such as SAP, Dynamics, and Oracle.
- **CMS Systems:** Including Adobe, Oracle, and other content management platforms.

The objective is to connect our bot with data repositories, minimizing data duplication as much as possible. These systems typically offer APIs, facilitating programmatic data access.

In this notebook, we plan to develop an agent capable of querying an API to retrieve information and effectively answer questions. We will maintain our focus on the COVID-19 theme and interact with the Open Disease Data API (https://disease.sh/).

In [1]:
import json
import requests
from time import sleep
from typing import Dict, List
from pydantic import BaseModel, Extra, root_validator

from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.requests import RequestsWrapper
from langchain.chains import APIChain
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

from common.callbacks import StdOutCallbackHandler
from common.utils import num_tokens_from_string, reduce_openapi_spec
from common.prompts import APISEARCH_PROMPT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string.replace("$","USD ")))


In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

llm_1 = AzureChatOpenAI(deployment_name="gpt-4-32k", temperature=0, max_tokens=2000, callback_manager=cb_manager)
llm_2 = AzureChatOpenAI(deployment_name="gpt-35-turbo-16k", temperature=0, max_tokens=1000)


## The Logic

By now, you must infer that the solution for an API Agent has to be something like: give the API specification as part of the system prompt to the LLM , then have an agent plan for the right steps to formulate the API call.<br>

Let's do that. But we must first understand the industry standards of Swagger/OpenAPI


## Introduction to OpenAPI (formerly Swagger)

The OpenAPI Specification, previously known as the Swagger Specification, is a specification for a machine-readable interface definition language for describing, producing, consuming and visualizing web services. Previously part of the Swagger framework, it became a separate project in 2016, overseen by the OpenAPI Initiative, an open-source collaboration project of the Linux Foundation.

OpenAPI Specification is an API description format for REST APIs. An OpenAPI file allows you to describe your entire API, including: Available endpoints (/users for example) and operations on each endpoint ( GET /users, POST /users), description, contact information, license, terms of use and other information.

### Let's get the OpenAPI (Swagger) spec from our desired API that we want to talk to

In [4]:
url = 'https://disease.sh/apidocs/swagger_v3.json'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    spec = response.json()
else:
    spec = None
    print(f"Failed to retrieve data: Status code {response.status_code}")


Let's see how big is this API specification:

In [5]:
# You can check the function "reduce_openapi_spec()" in utils.py
reduced_api_spec = reduce_openapi_spec(spec)

In [6]:
api_tokens = num_tokens_from_string(str(spec))
print("API spec size in tokens:",api_tokens)
api_tokens = num_tokens_from_string(str(reduced_api_spec))
print("Reduced API spec size in tokens:",api_tokens)

API spec size in tokens: 12855
Reduced API spec size in tokens: 8772


Sometimes it makes sense to reduce the size of the API Specs by using the `reduce_openapi_spec` function. It's optional.

## Question
Let's make a complicated question that requires two distinct API calls to different endpoints:

In [7]:
QUESTION = '''
What is to-date the amount of people tested in Argentina vs USA? 
Also tell me what is the continent with more covid deaths and what's the count as of today
'''

## Use a chain to convert the natural language question to an API request using the API specification in the prompt

We can use a nice chain in langchain called APIChain

In [8]:
# Most of APIs require Authorization tokens, so we construct the headers using a lightweight python request wrapper called RequestsWrapper
access_token = "ABCDEFG123456" 
headers = {"Authorization": f"Bearer {access_token}"}
requests_wrapper = RequestsWrapper(headers=headers)

**Note**: Notice that we are using GPT-3.5 (llm_2) below for this chain since it doesn't need too many instructions or reasoning

In [9]:
chain = APIChain.from_llm_and_api_docs(
    llm=llm_2,
    api_docs=str(reduced_api_spec),
    headers=headers,
    verbose=True,
    limit_to_domains=["https://disease.sh/"],
)


These are the prompts on the APIChain class (on to create the URL endpoint and the other one to use it and get the answer):

In [10]:
chain.api_request_chain.prompt.template

'You are given the below API Documentation:\n{api_docs}\nUsing this documentation, generate the full API url to call for answering the user question.\nYou should build the API url in order to get a response that is as short as possible, while still getting the necessary information to answer the question. Pay attention to deliberately exclude any unnecessary pieces of data in the API call.\n\nQuestion:{question}\nAPI url:'

In [11]:
chain.api_answer_chain.prompt.template

'You are given the below API Documentation:\n{api_docs}\nUsing this documentation, generate the full API url to call for answering the user question.\nYou should build the API url in order to get a response that is as short as possible, while still getting the necessary information to answer the question. Pay attention to deliberately exclude any unnecessary pieces of data in the API call.\n\nQuestion:{question}\nAPI url: {api_url}\n\nHere is the response from the API:\n\n{api_response}\n\nSummarize this response to answer the original question.\n\nSummary:'

In [13]:
try:
    printmd(chain.run(QUESTION))
except Exception as e:
    print(e)



> Entering new APIChain chain...
https://disease.sh/v3/covid-19/countries/Argentina,USA
<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>disease.sh | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-blo

The API call to retrieve the information about the number of people tested in Argentina and the USA resulted in a 502 Bad Gateway error. Therefore, we were unable to obtain the necessary data to answer the question.

As we have seen before in prior notebooks, a single chain cannot reason/observe/think, so it cannot figure out that it needs to call two endpoints in order to get the continents information. As you will see below, North America is NOT the continent with the highest amount of deaths.

## Creating a custom agent that uses the APIChain as a tool

To solve the avobe problem, we can build a REACT Agent that uses the APIChain as a tool to get the information. This agent will create as many calls as needed (using the chain tool) until it answers the question

In [14]:
class MyAPISearch(BaseTool):
    """APIChain as an agent tool"""
    
    name = "@apisearch"
    description = "useful when the questions includes the term: @apisearch.\n"

    llm: AzureChatOpenAI
    api_spec: str
    headers: dict = {}
    limit_to_domains: list = []
    verbose: bool = False
    
    def _run(self, query: str) -> str:
        
        chain = APIChain.from_llm_and_api_docs(
                            llm=self.llm,
                            api_docs=self.api_spec,
                            headers=self.headers,
                            verbose=self.verbose,
                            limit_to_domains=self.limit_to_domains
                            )
        try:
            sleep(2) # This is optional to avoid possible TPM rate limits
            response = chain.run(query)
        except Exception as e:
            response = e
        
        return response
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

Notice below that we are using GPT-35-Turbo-16k (llm_2) for the Tool and GPT-4-turbo (llm_1) for the Agent

In [15]:
tools = [MyAPISearch(llm=llm_2, api_spec=str(spec), limit_to_domains=["https://disease.sh/"])]
agent_executor = initialize_agent(tools, llm_1, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, agent_kwargs={'prefix':APISEARCH_PROMPT_PREFIX}, callback_manager=cb_manager)

In [16]:
# Uncomment if you want to see the prompt
# printmd(agent_executor.agent.llm_chain.prompt.template)

In [17]:
%%time 

#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue
        
printmd(response)

The user is asking for up-to-date information on the number of people tested for COVID-19 in Argentina and the USA. They also want to know which continent has the highest number of COVID-19 deaths and the current count. I will use the @apisearch tool to find this information.

Action: @apisearch
Action Input: COVID-19 testing numbers Argentina USA
The API search for COVID-19 testing numbers in Argentina and the USA did not return any results due to a technical error. I will attempt the search again.

Action: @apisearch
Action Input: COVID-19 testing numbers Argentina USA
The API search for COVID-19 testing numbers in Argentina and the USA did not return any results due to a technical error. I will attempt the search again.

Action: @apisearch
Action Input: COVID-19 testing numbers Argentina USA
The API search for COVID-19 testing numbers in Argentina and the USA did not return any results due to a technical error. I will now search for the continent with the highest number of COVID-19 

I'm sorry, but I was unable to retrieve the requested information due to technical issues. I recommend trying again later or seeking the information from a reliable source such as the World Health Organization or the Centers for Disease Control and Prevention.

CPU times: user 490 ms, sys: 23.7 ms, total: 514 ms
Wall time: 6min 5s


**Great!!** we have now an API Agent using APIChain as a tool, capable of reasoning until it can find the answer. And it is pretty fast as well.

## Simple APIs

What happens if the API is quite basic, meaning it's just a simple endpoint without a Swagger/OpenAPI definition? Let’s consider the following example:

[CountdownAPI](https://www.countdownapi.com/) is a streamlined version of the eBay API, available as a paid service. We can test it using their demo query, which does not require any Swagger or OpenAPI specification. In this scenario, our main task is to create a tool that retrieves the results. We then pass these results to an agent for analysis, providing answers to user queries, similar to our approach with the Bing Search agent.

An aspect we haven't discussed yet while constructing our API Agent using the APIChain tool is handling situations where either the API specification or the API call results are quite extensive. In such cases, we need to choose between using GPT-4-32k and GPT-4-Turbo.

In the example below, there is no API specification, but the response from the API is rather lengthy. For this scenario, we will employ GPT-4-32k.

In [18]:
# set up the request parameters
params = {
  'api_key': 'demo',
  'type': 'search',
  'ebay_domain': 'ebay.com',
  'search_term': 'memory cards'
}

# make the http GET request to Countdown API
api_result = requests.get('https://api.countdownapi.com/request', params)

num_tokens = num_tokens_from_string(str(api_result.json())) # this is a custom function we created in common/utils.py
print("Token count:",num_tokens,"\n")  

# print the first 2000 characters of JSON response from Countdown API
print(json.dumps(api_result.json())[:2000], "...")

Token count: 13474 

{"request_info": {"success": true, "demo": true}, "request_parameters": {"type": "search", "ebay_domain": "ebay.com", "search_term": "memory cards"}, "request_metadata": {"ebay_url": "https://www.ebay.com/sch/i.html?_nkw=memory+cards&_sacat=0&_dmd=1&_fcid=1"}, "search_results": [{"position": 1, "title": "128GB 256GB 1TB Micro SD Card Memory Card TF Card with Free Adapter High Speed", "epid": "364200951508", "link": "https://www.ebay.com/itm/364200951508", "image": "https://i.ebayimg.com/thumbs/images/g/o6sAAOSw8iBkJp06/s-l300.jpg", "condition": "Brand New", "seller_info": {"name": "lansuostore", "review_count": 14168, "positive_feedback_percent": 98.5}, "is_auction": false, "buy_it_now": false, "free_returns": true, "sponsored": true, "prices": [{"value": 3.88, "raw": "$3.88"}, {"value": 13.19, "raw": "$13.19"}], "price": {"value": 3.88, "raw": "$3.88"}}, {"position": 2, "title": "Sandisk Micro SD Card Memory 32GB 64GB 128GB 256GB 512GB 1TB Lot Extreme Ultra", "epi

So, the answer from this product query (the demo only works with 'memory cards' - you will need to sign up for their trial if you want to try any query with an API key), is about 16.5k tokens. When combined with the prompt, we won't have any other option than to use GPT-4-32k or GPT-4 turbo models. 

In [19]:
class MySimpleAPISearch(BaseTool):
    """Tool for simple API calls that doesn't require OpenAPI 3.0 specs"""
    
    name = "@apisearch"
    description = "useful when the questions includes the term: @apisearch.\n"

    api_key: str
    
    def _run(self, query: str) -> str:
        
        params = {
          'api_key': self.api_key,
          'type': 'search',
          'ebay_domain': 'ebay.com',
          'search_term': query
        }

        # make the http GET request to Countdown API
        api_result = requests.get('https://api.countdownapi.com/request', params)
        
        try:
            response = json.dumps(api_result.json())
        except Exception as e:
            response = e
        
        return response
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

In [20]:
tools = [MySimpleAPISearch(api_key='demo')]
agent_executor = initialize_agent(tools, llm_1, 
                                  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                                  agent_kwargs={'prefix':APISEARCH_PROMPT_PREFIX}, 
                                  callback_manager=cb_manager,
                                  handle_parsing_errors=True)

In [21]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run('what is the price for SanDisk memory cards? give me the links please') 
        break
    except Exception as e:
        response = str(e)
        continue

printmd(response)

The user is asking for the price of SanDisk memory cards and also wants the links to the sources of this information. I will use the @apisearch tool to find this information.
Action: @apisearch
Action Input: SanDisk memory card price
The search did not return any results. I should try again with slightly different search terms.
Action: @apisearch
Action Input: price of SanDisk memory cards
The search results returned a list of SanDisk memory cards available on eBay with their prices. I will extract the relevant information and provide the links as requested by the user.
Action: None
Action Input: None


Here are some SanDisk memory cards available on eBay along with their prices:

1. [Lot of 4 SanDisk Extreme Secure Digital - 32GB/100MB SDSDX Memory Card](https://www.ebay.com/itm/305319773806) - USD 19.95
2. [Lot 4 x SanDisk 32GB SDHC Class 4 SD Flash Memory Card Camera SDSDB-032G 128GB](https://www.ebay.com/itm/253863195301) - USD 24.99
3. [SanDisk Extreme A2 A1 190MB/s 256GB 128GB 64GB 32GB micro SD SDXC Card Lot C10](https://www.ebay.com/itm/276244979102) - USD 8.39 to USD 52.99
4. [Lot of 4 SanDisk Secure Digital - Ultra 64GB/140MB SDXC UHS-I Memory Card](https://www.ebay.com/itm/386454457401) - USD 25.95
5. [Sandisk SD Cards 16GB 32GB 64GB 128GB 256GB Extreme Pro Ultra Memory Cards lot](https://www.ebay.com/itm/324078167020) - USD 7.98 to USD 225.11

Please note that prices may vary based on the condition of the product, the seller, and other factors.

# Summary

In this notebook, we learned about how to create very smart API agents for simple or complex APIs that use Swagger or OpenAPI specifications.
We see, again, that the key to success is to use: Agents with Expert tools + GPT-4 + good prompts.

As homework, try to create a shopping assistant for Etsy e-commerce site using the following API spec: (you will need to register for free and create an API-Key)

- https://developers.etsy.com/documentation/
- https://www.etsy.com/openapi/generated/oas/3.0.0.json

# NEXT

The Next Notebook will guide you on how we stick everything together. How do we use the features of all notebooks and create a brain agent that can respond to any request accordingly.